# Práctico N° 2 Análisis de Grafos

Integrantes: Nicolás Benjamín Ocampo, Antonela Sambuceti

En este práctico, trabajaremos con un dataset extraído de Twitter. La idea es emplear los conceptos de grafos vistos en clase sobre un caso real de actualidad.

## Dataset

El dataset consiste en un conjunto de hilos de tweets, con un total de ~150000 tweets, extraídos entre Enero y Marzo de 2021. La temática de los mismos está referida a la vacunación contra el covid-19 en Argentina.

Pueden descargar el dataset del siguiente [link](https://drive.google.com/file/d/1X_qKsE8muAnom2tDX4sLlmBAO0Ikfe_G/view?usp=sharing).

### Campos

- **created_at:** Fecha del tweet
- **id_str:** ID del tweet
- **full_text:** Contenido del tweet
- **in_reply_to_status_id:** ID del tweet inmediatamente anterior en el hilo
- **in_reply_to_user_id:** Autor del tweet inmediatamente anterior en el hilo
- **user.id:** Autor del tweet
- **user_retweeters:** Lista de ID de usuarios que retweetearon el tweet
- **sentiment:** Etiquetado manual que indica el sentimiento o intención del tweet con respecto al tweet anterior en el hilo

## Configuración inicial

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import math
from cdlib import NodeClustering, evaluation, algorithms
import igraph as ig
import networkx as nx
import pickle

from nltk import (corpus, tokenize, download)
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
download("stopwords")
download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anto_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anto_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Cargamos el grafo

In [2]:
larg_G = nx.read_edgelist('larg_G.edgelist')

In [3]:
df_user = pd.DataFrame({'user':list(larg_G.nodes())})

In [4]:
deg_seq = np.array([k for v, k in larg_G.degree()])
df_user['degree'] = deg_seq

### Cálculo de Medidas de Centralidad

In [5]:
#generados grafo para libreria igraph
g_ig = ig.Graph.TupleList(larg_G.edges())
print(g_ig.summary())

IGRAPH UN-- 39800 93404 -- 
+ attr: name (v)


In [6]:
betweenness = g_ig.betweenness()
eigenvector = nx.eigenvector_centrality(larg_G,max_iter=1000)
pagerank = nx.pagerank(larg_G)
clustering_coefficient =  nx.clustering(larg_G)

In [11]:
df_user['betweenness'] = betweenness
df_user['eigenvector'] = list(eigenvector.values())
df_user['pagerank'] = list(pagerank.values())
df_user['Cws'] = list(clustering_coefficient.values())

df_user.describe().round(2)

,degree,betweenness,eigenvector,pagerank,Cws
count,39800.00,3.980000e+04,39800.00,39800.00,39800.00
mean,4.69,5.310058e+04,0.00,0.00,0.10
std,69.30,2.036286e+06,0.00,0.00,0.24
min,1.00,0.000000e+00,0.00,0.00,0.00
25%,1.00,0.000000e+00,0.00,0.00,0.00
50%,1.00,0.000000e+00,0.00,0.00,0.00
75%,3.00,2.133930e+03,0.00,0.00,0.00
max,8207.00,2.703364e+08,0.36,0.04,1.00


In [8]:
columns=['degree','eigenvector','pagerank','Cws', 'betweenness']
data = {}
for col in columns:
    top_users = df_user.nlargest(10, columns=[col])['user'].tolist()
    data[col] = top_users
pd.DataFrame(data)

,degree,eigenvector,pagerank,Cws,betweenness
0,252168075,130979339,252168075,564042719,252168075
1,130979339,73102744,130979339,146621283,854743297
2,73102744,252168075,73102744,1166543037159358464,756645318708588544
3,367933714,367933714,593189095,712335069453688832,342825036
4,593189095,2687724840,367933714,813028689743114241,771044267355766784
5,2687724840,593189095,2687724840,1580608093,342016328
6,931564592328781824,312708081,144929758,166990862,1267917522407510019
7,144929758,144929758,931564592328781824,1200475594410151936,1539379135
8,312708081,931564592328781824,312708081,475408248,3865357583
9,1077176953,959033548379508736,1077176953,201418332,926883779490312193


### Cálculo de Comunidades

#### Algoritmo de Louvain

Recalculamos las comunidades con el algoritmo de Louvain con resolution 2, para quedarnos con 2 grandes comunidades para analizar.

In [9]:
comms = algorithms.louvain(larg_G, resolution=2, randomize=False)

In [41]:
comm_df = pd.DataFrame({
    "communities": comms.communities,
    "comm_id": np.arange(0, len(comms.communities))
})

In [12]:
comm_df = comm_df.assign(comm_size=comm_df["communities"].apply(lambda c: len(c)))

## Actividades

### 4. Extracción de etiquetas
En el archivo etiquetas.csv están las etiquetas para un pequeño subconjunto de nodos. Podemos interpretar el valor de la etiqueta como la pertenencia a una determinada clase, donde los usuarios de una misma clase en general tienden a expresar apoyo entre sí.

In [13]:
DATASET_FILE = "dataset/etiquetas.csv"

dtypes = {
    'id_str': str,
    'full_text': str,
    'in_reply_to_status_id': str,
    'in_reply_to_user_id': str,
    'user.id': str}
    
df_etiq = pd.read_csv(DATASET_FILE, dtype=dtypes)
df_etiq= df_etiq.rename(columns={'user.id':'user'})

In [14]:
df_etiq.Clase.value_counts()

0    107
1     89
Name: Clase, dtype: int64

In [15]:
df_etiq = pd.merge(df_etiq, df_user, on='user',how='left')
df_etiq= df_etiq.dropna()

In [16]:
df_etiq.Clase.value_counts()

0    84
1    68
Name: Clase, dtype: int64

#### 4.1 Usuarios referentes de cada clase 
Utilizar alguna medida de centralidad calculada sobre el grafo de retweets.

In [17]:
#Top para clase 0
columns=['degree','eigenvector','pagerank','Cws', 'betweenness']
data = {}
for col in columns:
    top_users = df_etiq[df_etiq['Clase'] == 0].nlargest(5, columns=[col])['user'].tolist()
    data[col] = top_users
pd.DataFrame(data)

,degree,eigenvector,pagerank,Cws,betweenness
0,252168075,252168075,252168075,2947569093,252168075
1,510740590,510740590,510740590,140374618,510740590
2,145493922,145493922,43814969,172417422,973491850530164736
3,484349869,484349869,145493922,973491850530164736,927377684224708608
4,43814969,1272713534078619648,484349869,811607948,141508805


Obsevamos que el usuario 252168075, correspondiente a Norabar, que había sido seleccionado en el top10 de influencers en el enunciado número 2, también se encuentra en como principal influencer de la clase 0.

In [18]:
#Top para clase 1
columns=['degree','eigenvector','pagerank','Cws', 'betweenness']
data = {}
for col in columns:
    top_users = df_etiq[df_etiq['Clase'] == 1].nlargest(5, columns=[col])['user'].tolist()
    data[col] = top_users
pd.DataFrame(data)

,degree,eigenvector,pagerank,Cws,betweenness
0,73102744,73102744,73102744,910956876686381058,332305405
1,367933714,367933714,367933714,146631761,1241874768569274368
2,2687724840,2687724840,2687724840,1212718019849248768,837186873143488514
3,3421061763,3421061763,3421061763,1183974626,71009257
4,4160782817,4160782817,4160782817,2693592596,1183974626


Obsevamos que el usuario 73102744, correspondiente a Ernestorr, que había sido seleccionado en el top10 de influencers en el enunciado número 2, también se encuentra en como principal influencer de la clase 1.

#### 4.2 Utilizando los resultados del práctico anterior, determinar si los usuarios de cada clase forman parte de distintas comunidades

In [19]:
com_1 = list(comm_df["communities"][0])
com_2 = list(comm_df["communities"][1])

In [32]:
comm_df

,communities,comm_id,comm_size
0,"[2412575347, 130979339, 1171384413948710916, 1...",0,19388
1,"[2252264887, 252168075, 204502692, 3442008615,...",1,15486
2,"[480627466, 593189095, 810065724, 387199960, 5...",2,3097
3,"[121474081, 1249220367593410561, 1282168010070...",3,213
4,"[1216417610, 1304145009374101504, 111614038, 9...",4,123
...,...,...,...
351,[1151320289550188544],351,1
352,[1442638747],352,1
353,[1607146380],353,1
354,[715274658015260674],354,1


In [20]:
clase_0 = list(df_etiq[df_etiq['Clase'] == 0]['user'])
clase_1 = list(df_etiq[df_etiq['Clase'] == 1]['user'])

In [21]:
#Cantidad de usuarios de la clase 1 que se encuentran en la comunidad 2
len([x for x in clase_0 if x in com_2])

84

In [22]:
#Cantidad de usuarios de la clase 0 que se encuentran en la comunidad 1
len([x for x in clase_1 if x in com_1])

65

Todos los usuarios de la clase 0 están presentes en la comunidad 2 y casi todos los usuarios de la clase 1 están presentes en la comunidad 1.

In [23]:
#Exportamos df_etiq para punto 5
df_etiq.to_csv('etiq.csv')

In [52]:
#Exportamos 2 principales comunidades para punto 5
comm_1 = pd.DataFrame(com_1).rename(columns={0:'comm_1'})
comm_1.to_csv('comm_1.csv')
comm_2 = pd.DataFrame(com_2).rename(columns={0:'comm_2'})
comm_2.to_csv('comm_2.csv')

### Opcional: 
Reconstruir el archivo "etiquetas.csv". Para eso, hacer lo siguiente 
- Construir un grafo en donde los nodos sean usuarios, y donde los enlaces unan dos nodos si entre ellos hubo más respuestas de apoyo que de oposición.
- Extraer las dos componentes más grandes del grafo. Esos serán nuestros nodos etiquetados.